In [81]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import scipy
from scipy.stats import norm

In [82]:
import pandas as pd

In [83]:
from arch import arch_model

In [84]:
stock='AXISBANK.NS'
data=yf.download(stock,'2005-01-01','2023-05-05')
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2005-01-03,37.000000,38.200001,37.000000,38.040001,32.853146,1421010
2005-01-04,38.220001,38.400002,37.400002,37.939999,32.766781,1115825
2005-01-05,37.990002,37.990002,35.509998,36.849998,31.825396,1163085
2005-01-06,36.820000,38.000000,35.400002,37.029999,31.980858,1192610
2005-01-07,36.130001,38.380001,36.130001,38.020000,32.835869,1530260
...,...,...,...,...,...,...
2023-04-27,888.000000,890.000000,877.500000,881.400024,879.590454,15992383
2023-04-28,885.099976,890.049988,853.650024,860.000000,858.234375,23004117
2023-05-02,870.000000,876.400024,864.250000,870.650024,868.862549,12947953


In [85]:
returns=np.diff(data['Adj Close'])
returns

array([ -0.08636475,  -0.94138527,   0.15546227, ...,  10.62817383,
       -10.77783203,   6.08746338])

In [86]:
import torch
import torch.nn as nn

In [87]:
import pandas as pd
import numpy as np
import arch

In [88]:
returns.shape

(4524,)

In [89]:
def garchquantile(returns):
  # Define the GARCH model
  garch_model = arch.arch_model(returns, vol='Garch', p=10, q=1)

# Fit the GARCH model
  garch_result = garch_model.fit(disp='off')

# Generate the GARCH volatility forecasts
  garch_forecasts = garch_result.forecast(start=0)

# Extract the conditional volatility at the last observation
  forecasted_volatility = np.sqrt(garch_forecasts.variance.values[-1])

# Calculate VaR for the 95th quantile
  confidence_level = 0.05  # 95% confidence level
  z_score = norm.ppf(confidence_level)
  VaR = -z_score * forecasted_volatility
  return VaR

In [90]:
predictions=np.zeros(453)
for i in range(453):
  predictions[i]=(garchquantile(returns[0:4071+i]))
predictions



Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to g

array([16.25679601, 15.82156202, 15.47099717, 15.40359994, 15.25545766,
       14.97725655, 15.97773789, 15.58866079, 15.17605336, 14.79918606,
       15.67240696, 15.7152227 , 15.24518804, 15.17728605, 14.75672064,
       17.30220344, 17.0831491 , 16.76697129, 16.62144104, 16.81278093,
       17.63499   , 18.10634481, 17.68101795, 17.52753616, 17.90364868,
       17.57732821, 17.08022664, 16.6691549 , 16.23354495, 15.80875223,
       15.512785  , 15.11621536, 15.68625632, 15.36287941, 15.41338851,
       15.25816452, 15.08392596, 14.84521112, 18.77416399, 18.28180612,
       18.38392999, 17.93709796, 17.5624824 , 17.10441292, 17.36525225,
       17.08476097, 16.71763736, 16.28125923, 15.9971313 , 15.60931546,
       15.7540399 , 15.42619892, 15.64763723, 15.22470564, 15.05852491,
       17.18090882, 17.56924174, 17.09979306, 16.90525297, 16.67371072,
       17.28904812, 16.82864346, 16.86839241, 16.46498385, 16.08891625,
       15.6697121 , 15.35010074, 15.05646784, 15.12058999, 14.76

In [91]:
ans=returns[4071:]
ans.shape

(453,)

In [92]:
error=predictions-ans
error

array([ 1.82496549e+01,  1.12877485e+01,  2.39406627e+01,  2.24284413e+01,
        9.02779652e+00, -2.11154960e+00,  1.66752477e+01,  1.15033336e+01,
        1.81653112e+01,  3.06921914e+01,  2.50887278e+01,  1.47187627e+01,
        6.62610845e+00,  1.42804965e+01,  3.92688666e+01,  2.60707581e+01,
        2.27130319e+01,  2.50871007e+01,  4.41520325e+00, -5.74914378e-01,
        2.48929908e+00,  2.25902315e+01,  2.61008788e+01,  2.92975784e+00,
        1.18753284e+01,  1.83246427e+01,  2.05179097e+01,  1.41282613e+01,
        1.41410767e+01,  2.10399534e+01,  1.78543988e+01,  2.92157027e+01,
        1.98214492e+01,  5.14950051e+00,  2.26873143e+01,  7.98423874e+00,
        9.05548357e+00, -1.66917762e+01,  1.63329408e+01,  5.92603220e+00,
        1.45974310e+01,  2.26203865e+01,  1.47724433e+01,  3.00082215e+01,
        1.05396297e+01,  2.16185744e+01,  1.85112775e+01,  1.02527558e+01,
        1.89863891e+01,  4.44940335e+00,  1.10210077e+01,  2.68851223e+01,
        1.61458672e+01,  

In [93]:
loss=0.0
for i in range(error.shape[0]):
  if(error[i]>0):
    loss=loss+0.95*error[i]
  else:
    loss=loss+(-1)*error[i]*(0.05)
loss/error.shape[0]

19.407964899825483